### Instagram posts retriever

In [5]:
from datetime import datetime, timedelta
from itertools import dropwhile, takewhile
import time
import os
import instaloader
import pandas as pd
import numpy as np
import re
from unidecode import unidecode
from ftfy import fix_text

### Create dataframe scraping all pages

In [6]:
# Save all
def update_db(users: list):
    L = instaloader.Instaloader()
    for user in users:
        profile = instaloader.Profile.from_username(L.context, user)
        posts = profile.get_posts()
        SINCE = datetime.today()
        UNTIL = SINCE - timedelta(days=15)
        for post in takewhile(lambda p: p.date > UNTIL, dropwhile(lambda p: p.date > SINCE, posts)):
            # L.download_post(post, user)
            print(post.date)
            print(post.caption)
            print(post.shortcode)
            time.sleep(2)
        time.sleep(30)

In [7]:
USERNAMES = ['angelo_mai_roma']#, 'forte_antenne', 'nuur.xyz']
update_db(USERNAMES)

2023-09-08 13:38:25
@angelo_mai_roma è il tuo teatro e ha ideato una FORMULA MAGICA per partecipare e sostenere la sua vita teatrale: 
✨ 5 spettacoli - 40 € + Bag
✨10 spettacoli - 80 € + 
  T-shirt 
✨15 spettacoli - 120 € + Bag + T-Shirt 
Ti aspetta MAGÌA: un anno di teatro, 24 compagnie, 3 laboratori annuali per tutte le età, 5 produzioni @bluemotion.theatre / Angelo Mai 
Potrai acquistarla già dal 9 al 13 settembre durante lo spettacolo Sogno Creatore di @giorgina_pi 
[Biglietti ancora disponibili per il 10, 12 e 13 settembre/ link in bio]
.
.
.
.
.
.
.
.
#angelomai #teatro #bluemotion #giorginapi #formulamagica #magia #teatroroma #eventiroma #theatre
Cw7sujMNsAw
2023-09-05 16:56:05
Il 12/09 h 18.30 presentiamo 𝗙𝗼𝘁𝗼𝗿𝗼𝗺𝗮𝗻𝘇𝗮 un progetto di Giorgina Pi a cui teniamo molto. Insieme a noi ci sarà Barbara Leda Kenny.  𝗙𝗼𝘁𝗼𝗿𝗼𝗺𝗮𝗻𝘇𝗮 è un laboratorio gratuito per donne dai  65 anni in su. Il suo obiettivo è costruire un fotoromanzo della propria vita.
Un oggetto cartaceo, un capolavoro persona

In [3]:
# Delete all files except .txt, .ipynb and .py
for root, dirs, files in os.walk(".", topdown=False):
    for name in files:
        if not name.endswith(".txt") and not name.endswith(".ipynb") and not name.endswith(".py"):
            os.remove(os.path.join(root, name))

In [8]:
# Create a dataframe with all the captions
def create_db():
    df = pd.DataFrame(columns=['date', 'caption', 'location'])
    for root, dirs, files in os.walk(".", topdown=False):
        for name in files:
            # Delete all files that are not txt in all subdirectories
            if not name.endswith(".txt") and not name.endswith(".ipynb"):
                os.remove(os.path.join(root, name))
            # Otherwise
            elif name.endswith(".txt"):
                with open(os.path.join(root, name), 'r', encoding='utf-8') as f: # Maybe an integration with chardet would be better
                    caption = f.read()
                    caption = fix_text(caption)
                    caption = unidecode(caption)
                    caption = re.sub(r'\s+', ' ', caption)
                    caption = caption.lower()
                    caption = re.sub(r'\W+', ' ', caption)
                    date = re.findall(r'\d{4}-\d{2}-\d{2}', name)[0]
                    location = root[2:]
                    df = pd.concat([df, pd.DataFrame({'date': [date], 'caption': [caption], 'location': [location]})], ignore_index=True)
    df.drop_duplicates(subset=['caption'], inplace=True)
    
    # Check if the caption refers to an event
    whitelist = ['dj set', 'djset', 'concerto', 'live set', 'liveset', 'line up', 'lineup', 'b2b']
    blacklist = ['annullato', 'annullata', 'annullare', 'cancellato', 'cancellata', 'posticipato', 'posticipata', 'sospeso', 'sospesa']
    df['event'] = df['caption'].str.contains('|'.join(whitelist)) & ~df['caption'].str.contains('|'.join(blacklist))
    return df
df = create_db()
df.head()

,date,caption,location
0,2023-07-03,if i can t dance it s not my revolution dopo ...,angelo_mai_roma
1,2023-07-10,venerdi 14 luglio merende presenta superorgani...,angelo_mai_roma
2,2023-07-19,venerdi 21 luglio superorganismo last dance sp...,angelo_mai_roma
3,2023-07-28,ci rivediamo il 9 settembre con sogno creatore...,angelo_mai_roma
4,2023-08-28,la seconda sezione civile del tribunale di rom...,angelo_mai_roma


In [10]:
df.to_csv('db.csv', index=False)

### Single posts handler

In [2]:
def ig_post_handler(link):
    L = instaloader.Instaloader()
    post = instaloader.Post.from_shortcode(L.context, link.split('/')[-2])
    return post

post = ig_post_handler('https://www.instagram.com/p/CmrVxmut9DQ/?igshid=NzZhOTFlYzFmZQ==')

In [22]:
def clean_text(text):
    text = fix_text(text)
    text = re.sub(r'\s+', ' ', text)
    # text = unidecode(text)
    # text = text.lower() # Since gpt better understands uppercase letters, I'll leave them
    text = re.sub(r'(?<!\d)-(?!\d)', ' ', text) # Delete all special characters a part from the ones dashes that are preceded and followed by numbers
    return text

In [4]:
post.shortcode

'CmrVxmut9DQ'

In [23]:
caption = clean_text(post.caption)
caption

"VOLTA 06.01.2023 Il 2023 inizia con una nuova festa NUUR che questa volta appare nel ventre dell'Esquilino, in un antico deposito sotterraneo. In console Brando Lupi, ambasciatore romano dell'hypnotic techno e Close Listen, produttore trap che torna dietro la consolle per un inedito dj set techno. Brando Lupi @brando.lupi Close Listen @closelisten Dj Ostia @kodacci Diablo Viola @__lucaviola__ Dalle 22.30 fino a tardi Club 79 (Ex Micca Club) Via Pietro Micca 7, Roma TICKET: in prevendita 15€ + drink, alla porta 15€ LINK IN BIO"